### **Options Database Download WRDS**

In [2]:
import wrds 
import pandas as pd
import numpy as np
from datetime import date

**Inputs**

You can find SEC ID's here: https://wrds-www.wharton.upenn.edu/search/company-search/code-lookup/?product_id=274&attribute_type_map=

In [33]:
# Start/End Year
start_year = 2020
end_year = 2023

mag7 = {
    "AAPL": 101594,
    "AMZN": 101310,
    "GOOG": 203876,
    "META": 154402,
    "MSFT": 107525,
    "NVDA": 108321,
    "TSLA": 143439
}

secids = list(mag7.values()) + [108105,102442] # SPX, SML

# Limit to 1M Options
expiration_max = 31
expiration_min = 27

**Query**

Options prices for different years are stored in different tables. The code below just applies a union all to the queries.

In [37]:
base_query = """
SELECT
    date, secid, symbol,
    exdate, cp_flag, strike_price,
    best_bid, best_offer, impl_volatility,
    delta, gamma, vega, theta
FROM 
    optionm_all.opprcd{year} as o
WHERE
    o.secid IN {secids}
    AND (o.exdate - o.date <= {expiration_max})
    AND (o.exdate - o.date >= {expiration_min})
"""

expiration_max = 31
expiration_min = 27

# Build the query
queries = [
    base_query.format(year=year, secids=tuple(secids), expiration_max=expiration_max, expiration_min=expiration_min)
    for year in range(start_year,end_year+1)
]

# Join subqueries
q = " UNION ALL ".join(queries)

**Run**

In [39]:
conn = wrds.Connection()
data = conn.raw_sql(q)

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [45]:
# data.to_parquet('../data/wrds/options_prices_2020_2023.parquet')

**Import**

In [46]:
# pip install pyarrow
df = pd.read_parquet('../data/wrds/options_prices_2020_2023.parquet')
df.head()

,date,secid,symbol,exdate,cp_flag,strike_price,best_bid,best_offer,impl_volatility,delta,gamma,vega,theta
0,2023-01-03,101310.0,AMZN 230203C100000,2023-02-03,C,100000.0,1.13,1.24,0.518193,0.180449,0.020279,6.573099,-20.64159
1,2023-01-03,101310.0,AMZN 230203C101000,2023-02-03,C,101000.0,0.99,1.1,0.517512,0.163279,0.019051,6.167326,-19.3229
2,2023-01-03,101310.0,AMZN 230203C102000,2023-02-03,C,102000.0,0.89,0.98,0.51952,0.148764,0.017848,5.791956,-18.22698
3,2023-01-03,101310.0,AMZN 230203C103000,2023-02-03,C,103000.0,0.79,0.89,0.522221,0.135744,0.016688,5.445606,-17.20464
4,2023-01-03,101310.0,AMZN 230203C104000,2023-02-03,C,104000.0,0.69,0.79,0.522084,0.122306,0.015528,5.073944,-15.98841
